In [1]:
import AdventOfCode

In [2]:
input <- dayLines 4

In [3]:
import qualified Data.Attoparsec.ByteString as A
import qualified Data.Attoparsec.ByteString.Char8 as C

head input

"[1518-09-17 23:48] Guard #1307 begins shift"

In [5]:
import Data.List

sortedInput = sort input

In [18]:
{-# LANGUAGE OverloadedStrings #-}

import Control.Applicative ((<|>))


data Event
    = Guard Int
    | FallsAsleep
    | WakesUp
    deriving (Eq, Show)

data Timestamp = Timestamp
    { year
    , month
    , day
    , hour
    , minute :: Int
    } deriving (Eq, Show)

data Line = Line Timestamp Event deriving (Eq, Show)

parseTimestamp = do
    C.char '['
    y <- C.decimal
    C.char '-'
    m <- C.decimal
    C.char '-'
    d <- C.decimal
    C.space
    h <- C.decimal
    C.char ':'
    m <- C.decimal
    C.char ']'
    pure $ Timestamp y m d h m

parseChange = do
    A.string "Guard #"
    no <- C.decimal
    A.string " begins shift"
    pure $ Guard no 

parseEvent = parseChange <|> (A.string "falls asleep" *> pure FallsAsleep) <|> (A.string "wakes up" *> pure WakesUp)

parseLine = (,) <$> (parseTimestamp <* " ") <*> parseEvent

parsed parseTimestamp "[1518-03-03 00:02]"

parsedInput = map (parsed parseLine) sortedInput

Timestamp {year = 1518, month = 2, day = 3, hour = 0, minute = 2}

In [28]:
import qualified Data.Map.Strict as Map

data GuardState
    = GuardState
    { currentGuard
    , fellAsleep :: Int
    } deriving (Eq, Show)

processInput :: (GuardState, Map.Map Int [Int]) -> (Timestamp,Event) -> (GuardState, Map.Map Int [Int])
processInput (gs, mp) (ts,event) = case event of
    Guard i -> (gs { currentGuard = i }, mp)
    FallsAsleep -> (gs { fellAsleep = minute ts }, mp)
    WakesUp -> (gs, Map.insertWith (++) (currentGuard gs) [fellAsleep gs..(minute ts - 1)] mp)

state = foldl' processInput (GuardState 0 0,Map.empty) parsedInput

In [34]:
import Data.Ord (comparing)
import Data.Function (on)

stateList = Map.toList $ snd state

sleepiest = maximumBy (compare `on` (length . snd) ) stateList

freqs :: Ord a => [a] -> Map.Map a Int
freqs as = foldl' (\m a -> Map.insertWith (+) a 1 m) Map.empty as

fst sleepiest 
fst $ maximumBy (compare `on` snd) $ Map.toList $ freqs $ snd sleepiest

3209*32

3209

32

102688

In [41]:
sleepFreqs = Map.map (maximumBy (compare `on` snd) . Map.toList . freqs) $ snd state
maximumBy (compare `on` (snd . snd) ) $ Map.toList sleepFreqs
1459 * 39

(1459,(39,18))

56901